In [1]:
import numpy as np
import os
os.getcwd()

data = np.load("../data/processed/supervised_train.npz")
X, y = data["X"], data["y"]

print("X shape:", X.shape)          # Expect (N, 12, 8, 8)
print("y shape:", y.shape)          # Expect (N,)
print("dtype:", X.dtype, y.dtype)
print("action_space:", y.max() + 1, "max index:", y.max())

X shape: (348113, 12, 8, 8)
y shape: (348113,)
dtype: float32 int64
action_space: 4661 max index: 4660


In [1]:
import sys
from pathlib import Path
import numpy as np
import torch
import os

os.chdir("C:/Users/salva/.jupyter/cs372-chess-bot")
repo_root = Path("C:/Users/salva/.jupyter/cs372-chess-bot").resolve()
sys.path.insert(0, str(repo_root / "src"))

from chess_engine.action_encoding import ACTION_SIZE
from chess_engine.state_encoding import board_to_tensor
from chess_engine.ChessGame import ChessGame
from chess_nnet.NNetWrapper import NNetWrapper
from core.MCTS import MCTS
from envs.chess_gym import GymChessEnv

# 1. DATASET 
print("\n[1] Dataset Integrity")
try:
    train_data = np.load("data/processed/supervised_train.npz")
    val_data = np.load("data/processed/supervised_val.npz")
    test_data = np.load("data/processed/supervised_test.npz")
    
    for name, data in [("train", train_data), ("val", val_data), ("test", test_data)]:
        X, y = data["X"], data["y"]
        print(f"{name:6} | X: {X.shape} {X.dtype} | y: {y.shape} {y.dtype}")
        assert X.ndim == 4 and X.shape[1:] == (12, 8, 8), f"Bad X shape: {X.shape}"
        assert y.ndim == 1, f"Bad y shape: {y.shape}"
        assert len(y) == X.shape[0], "X and y length mismatch"
        assert np.isfinite(X).all() and np.isfinite(y).all(), "NaN/Inf detected"
        assert (y >= 0).all() and (y < ACTION_SIZE).all(), f"Bad label range: [{y.min()}, {y.max()}]"
        assert X.min() >= 0 and X.max() <= 1, f"X not normalized: [{X.min()}, {X.max()}]"
    print("✓ All datasets valid\n")
except Exception as e:
    print(f"✗ Dataset check failed: {e}\n")

# 2. GAME ENGINE
print("[2] Game Engine")
try:
    game = ChessGame()
    print(f"✓ ChessGame created")
    print(f"  Board size: {game.getBoardSize()}")
    print(f"  Action space: {game.getActionSize()}")
    
    board = game.getInitBoard()
    print(f"  Initial board type: {type(board).__name__}")
    
    canon = game.getCanonicalForm(board, 1)
    print(f"  Canonical form type: {type(canon).__name__}")
    
    valid_moves = game.getValidMoves(board, 1)
    print(f"  Valid moves shape: {valid_moves.shape}, count: {int(valid_moves.sum())}")
    assert valid_moves.sum() == 20, f"Initial position should have 20 moves, got {int(valid_moves.sum())}"
    
    game_ended = game.getGameEnded(board, 1)
    print(f"  Game ended (initial): {game_ended}")
    assert game_ended == 0, "Initial position should not be terminal"
    
    print("✓ Game engine OK\n")
except Exception as e:
    print(f"✗ Game engine check failed: {e}\n")

# 3. STATE ENCODING
print("[3] State Encoding")
try:
    import chess
    board = chess.Board()
    tensor = board_to_tensor(board)
    print(f"✓ board_to_tensor works")
    print(f"  Shape: {tensor.shape}, dtype: {tensor.dtype}")
    print(f"  Min/max: [{tensor.min():.2f}, {tensor.max():.2f}]")
    assert tensor.shape == (12, 8, 8), f"Bad tensor shape: {tensor.shape}"
    assert tensor.dtype == np.float32, f"Bad dtype: {tensor.dtype}"
    assert np.isfinite(tensor).all(), "NaN/Inf in tensor"
    print("✓ State encoding OK\n")
except Exception as e:
    print(f"✗ State encoding check failed: {e}\n")

# 4. NEURAL NETWORK
print("[4] Neural Network")
try:
    game = ChessGame()
    nnet = NNetWrapper(game)
    print(f"✓ NNetWrapper created")
    print(f"  Device: {nnet.device}")
    print(f"  Network: {type(nnet.nnet).__name__}")
    
    # Test prediction
    board = game.getInitBoard()
    canon = game.getCanonicalForm(board, 1)
    tensor = board_to_tensor(canon).astype(np.float32)
    
    pi, v = nnet.predict(tensor)
    print(f"✓ predict() works")
    print(f"  Policy shape: {pi.shape}, dtype: {pi.dtype}")
    print(f"  Value: {v:.4f}, dtype: {type(v).__name__}")
    
    assert pi.shape == (ACTION_SIZE,), f"Bad policy shape: {pi.shape}"
    assert np.isfinite(pi).all(), "NaN/Inf in policy"
    assert np.isfinite(v), "NaN/Inf in value"
    assert pi.min() >= 0 and pi.max() <= 1, f"Policy not normalized: [{pi.min()}, {pi.max()}]"
    
    print("✓ Neural network OK\n")
except Exception as e:
    print(f"✗ Neural network check failed: {e}\n")

# 5. MCTS CHECKS
print("[5] Monte Carlo Tree Search")
try:
    game = ChessGame()
    nnet = NNetWrapper(game)
    args = type("Args", (), {"numMCTSSims": 5, "cpuct": 1.0})()
    mcts = MCTS(game, nnet, args)
    print(f"✓ MCTS created")
    
    board = game.getInitBoard()
    canon = game.getCanonicalForm(board, 1)
    
    probs = mcts.getActionProb(canon, temp=1)
    print(f"✓ getActionProb() works")
    print(f"  Shape: {probs.shape}, dtype: {type(probs).__name__}")
    print(f"  Sum: {np.sum(probs):.6f}, valid actions: {np.sum(probs > 0)}")
    
    assert probs.shape == (ACTION_SIZE,), f"Bad probs shape: {probs.shape}"
    assert np.isclose(np.sum(probs), 1.0, atol=1e-5), f"Probs don't sum to 1: {np.sum(probs)}"
    assert np.isfinite(probs).all(), "NaN/Inf in probs"
    
    print("✓ MCTS OK\n")
except Exception as e:
    print(f"✗ MCTS check failed: {e}\n")

# 6. GYM ENVIRONMENT CHECKS
print("[6] Gym Environment")
try:
    env = GymChessEnv()
    print(f"✓ GymChessEnv created")
    
    obs, info = env.reset()
    print(f"✓ reset() works")
    print(f"  Obs shape: {obs.shape}, dtype: {obs.dtype}")
    
    valid = env.game.getValidMoves(env._board, env._player)
    action = int(np.argmax(valid))
    
    obs2, reward, term, trunc, info = env.step(action)
    print(f"✓ step() works")
    print(f"  Reward: {reward}, terminated: {term}, truncated: {trunc}")
    
    assert obs.shape == (12, 8, 8), f"Bad obs shape: {obs.shape}"
    assert isinstance(reward, (int, float, np.number)), f"Bad reward type: {type(reward)}"
    assert isinstance(term, (bool, np.bool_)), f"Bad term type: {type(term)}"
    
    print("✓ Gym environment OK\n")
except Exception as e:
    print(f"✗ Gym environment check failed: {e}\n")



[1] Dataset Integrity
train  | X: (348113, 12, 8, 8) float32 | y: (348113,) int64
val    | X: (116038, 12, 8, 8) float32 | y: (116038,) int64
test   | X: (116038, 12, 8, 8) float32 | y: (116038,) int64
✓ All datasets valid

[2] Game Engine
✓ ChessGame created
  Board size: (8, 8)
  Action space: 4672
  Initial board type: Board
  Canonical form type: Board
  Valid moves shape: (4672,), count: 20
  Game ended (initial): 0
✓ Game engine OK

[3] State Encoding
✓ board_to_tensor works
  Shape: (12, 8, 8), dtype: float32
  Min/max: [0.00, 1.00]
✓ State encoding OK

[4] Neural Network
✓ NNetWrapper created
  Device: cpu
  Network: ChessResNet
✓ predict() works
  Policy shape: (4672,), dtype: float32
  Value: -0.0191, dtype: float
✓ Neural network OK

[5] Monte Carlo Tree Search
✓ MCTS created
✓ getActionProb() works
  Shape: (4672,), dtype: ndarray
  Sum: 1.000000, valid actions: 3
✓ MCTS OK

[6] Gym Environment
✓ GymChessEnv created
✓ reset() works
  Obs shape: (12, 8, 8), dtype: float32
✓